In [19]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os
import pprint
from dotenv import load_dotenv
load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = Chroma(persist_directory="./jonhWick_db", embedding_function=embeddings, collection_name="doc_jonhWick")


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('MY_ENV_VAR')

Esta tecnica intenta evitar el sobre consumo de tokens, y por lo tanto, dar demsiado contexto al LLM. Para la creación de un RAG, siempre hay que tener en mente dos puntos:

 - Context Windows: Cuanto más documentos obtengamos de la vectore store, más información tendrá el LLM para dar una buena respuesta.
 - Recall: Cuanto más documentos sean recuperados de la vectore store, la probabilidad de obtener chunks irrevalantes es mayor y por lo tanto, el recall disminuye.

Parece que no hay solución para este problema. Cuando aumentamos una de las métricas, la otra parece estar destinada a disminuir. ¿Estamos seguros de eso?

Es aqui cuando se presenta esta tecnica, compression retriever, focalizandonos en la técnica del reranking. Digamos que esta técnica consta de dos pasos bien diferenciados:

 - Obtener una buena cantidad de docs relevantes en función del input/pregunta. Normlamente fijamos los K más relevantes.
 - Recalcular cuales de estos documentos son realmente relevantes.


Para el primer paso, se usa lo que se conoce como Bi-encoder, que no es nada más que lo que solemos usar para hacer un RAG básico. Vectorizar nuestros documentos. vectorizar la quuery y calcular la similitud con cualquier métrica de nuestra elección.

El segundo paso es algo diferente a lo que estamos habituados a ver. Este recalculo/reranking, aparecen los reranking model o cross-encoder. Estos modelos esperan como input dos documentos/textos devolviendo una score de similitud entre el par. 

Si uno de estos dos inputs es la query y el otro es un chunk, podemos calcular la similitud entre ambos. Pero Damián, eso ya lo hacemos cuando aplicamos los bi-encoders.

Esto es totalmente cierto, pero hay una caracteristica clave. Y es que este tipo de técnica mejora el resultado de la similitud entre dos textos.

De acuerdo, funciona mejor, entonces porque no lo usamos directamente con todos los chunks, en vez de solo con los K mejores. Porque sería terriblemente costoso en tiempo y en dinero/computación. Por ello hacemos un primer filtro de los chunks más cercanos en similitud con la query, reduciendo el uso del modelo de reranking a tan solo K veces. 

De hecho este tipo de modelos recogen los dos inputs comentados previamente, y devuelve tan solo un numero entre 0 y 1. De hecho, para ser más especificos, esto se conoce como modelos "Cross-Encoder". Estos modelos no devuelven un sentences embedding, por lo tanto su uso está muy restringido a unos casos de uso.


Para entender mejor la acquitectura de este tipo de modelos, veamos un ejemplo visual.




Volviendo de nuevo al coste computacional y de tiempo, si se aplicase directamente los cross-encoders, pensar que con cada nueva consulta, se debería de calcular la similitud de la query con cada uno de los documentos. Algo que no es nada optimo.

En cambio usando los Bi-encoder, la representación vectorial de los documentos es la misma para cada nueva consulta. 

Tenemos entonces un metodo bastante superior que es costoso de ejecutar, y por otro lado, otro metodo que funciona bien pero que no tiene un gran costo computacional con cada nueva consulta. Todo esto finaliza con la conclusión de unificar estos dos métodos para un mejor RAG. Y esto es conocido como los contextual Compression con el método de reranking. 


https://www.sbert.net/examples/applications/cross-encoder/README.html 

## To create a compression retriever, we need a base retriever first. In this case I will use the naive retriever (the simplest)

In [3]:
naive_retriever = vectordb.as_retriever(search_kwargs={ "k" : 10})

In [5]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')

compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

## We are going to do a Naive RAG.

## Remember:

- R -> Retrieval
- A -> Augmented
- G -> Generation

# Retrieval

In [6]:
# We have already created the retriever object
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x00000170F1131C30>, top_n=3, model='rerank-english-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000170DD6F7DC0>, search_kwargs={'k': 10}))

# Augmented

In [7]:
from langchain_core.prompts import ChatPromptTemplate

TEMPLATE = """\
You are happy assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

# Generation

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

## Finally, we are going to create a Rag Parent doc Retrieval. For that, we are going to use LCEL (LangChain Expression Language)
If you want to learn more about LCEL, check this good tutorial: https://www.youtube.com/watch?v=O0dUOtOIrfs

In [10]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compression_retriever })
output_parser = StrOutputParser()


compressor_retrieval_chain = setup_and_retrieval | rag_prompt | chat_model | output_parser


compressor_retrieval_chain.invoke( "Did people generally like John Wick?")

'Yes, people generally liked John Wick.'

In [11]:
compressor_retrieval_chain.invoke("What are the reviews with a score greater than 7?")

"I'm sorry, I don't know the answer."

In [12]:
from langchain.globals import set_verbose, set_debug

set_debug(True)
compressor_retrieval_chain.invoke("What are the reviews with a score greater than 7 and say bad things about the movie?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are the reviews with a score greater than 7 and say bad things about the movie?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context>] Entering Chain run with input:
{
  "input": "What are the reviews with a score greater than 7 and say bad things about the movie?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context> > 3:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What are the reviews with a score greater than 7 and say bad things about the movie?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context> > 3:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What are the reviews with a score greater than 7 and say bad things about the movie?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<question,context>] s] Exiting Chain run wi

"I don't know."

### Finaly, the query is : talk bad about the movie and the filter is "Rating" greater than 7